Description of CABARET 

## Importation

In [1]:
import numpy as np
import sys
import time
import os
import config.muttconfig
import config.input


from shock import shock
from total import total
from heatflux import heatflux
import rebuilding_setup as setup
from atmUS76 import Atmosphere
from config.input import pr, L, T_w

atmosphere(0.00046370449579550513, 42.52011324335111, 266.9675706383899)


## Free stream condition

In [2]:
#Free stream conditions
M_1 = 28.56
reff = 0.48
alt = 70 

p_1 = Atmosphere(alt)[0]
T_1 = Atmosphere(alt)[1]

print(p_1, T_1, M_1)

5.219719752495632 224.80071443976283 28.56


mutation++ mixture setup 

In [3]:
mix = setup.setup_mpp('/Users/jeannelonglune/Desktop/memoire/pyCabaret/input.in')


In [4]:
# Specify module options TO DO : develop on modul options
options1 = {"ratio": 0.2,  #limit bf infinite loop is 0.33 #initial guess rho1/rho2
        "robust": "Yes"}

    #results for 0.33, 0.3, 0.2, 0.1 & Yes and No : Qw = -32753.33 W/m2 for any size
    #not the same as in cabaret initial
    #=> feel like I've an heaviside function... 

## Shock : Rankine-Hugoniot

We solve the following system of equations known as the Rankine-Hugoniot relations

$\rho_{1}v_{1} = \rho_{2}v_{2}$,

$P_{1} + \rho_{1}v_{1}^{2} = P_{2} + \rho_{2}v_{2}^{2}$,

$(\rho_{1}E_{1} + P_{1})v_{1} = (\rho_{2}E_{2} + P_{2})v_{2}$.

The variable names are quite self-explanatory.

In practice, we solve for the density ratio $\rho_{1}/\rho_{2}$, while updating the temperature to match the total energy and the corresponding pressure.

In [5]:
preshock_state = [T_1,p_1,M_1]

start_time = time.time()
T2,p2,v2 = shock(preshock_state,mix,options1)
end_time = time.time()
exec_time = end_time - start_time

print('T2 = '+ "{:.2f}".format(T2)+' K;', 'p2 = '+ "{:.2f}".format(p2)+' Pa;', 'v2 = '+ "{:.2f}".format(v2)+' m/s')
print('Execution time = '+"{:.4f}".format(exec_time), ' seconds = '+"{:.4f}".format(exec_time/60), ' minutes')

T2 = 6859.21 K; p2 = 5642.32 Pa; v2 = 465.98 m/s
Execution time = 0.0545  seconds = 0.0009  minutes


## Total quantities ##
Computing the total or stagnation quantities of a given state (temperature, pressure and velocity) is equivalent to doing a reservoir computation. Isentropic relations to deccelerate the flow to stagnant conditions. We solve the same equations.

Now, the variables we solve for are proportional to the state from which we want to compute the total quantities, in this case $T_{2}, P_{2}, v_{2}$. This is why we choose to initialize these variables with 1.0.

In [6]:
options2 = {"pressure": 1.0, 
           "temperature": 1.0,
           "robust": "No"}

In [9]:
mix.equilibrate(T2, p2)
ht2 = mix.mixtureHMass()
print(ht2)

36804532.37466799


## Heatflux computation : Fay and Riddel 

In [11]:
start_time = time.time()
qw = heatflux(mix, pr, L, p_1, p2, T2, ht2, reff, T_w)
end_time = time.time()

exec_time = end_time - start_time

print('Qw = '+ "{:.2f}".format(qw)+' W/m2')
print('Execution time = '+"{:.4f}".format(exec_time), ' seconds = '+"{:.4f}".format(exec_time/60), ' minutes')

Qw = 1597900.16 W/m2
Execution time = 0.0023  seconds = 0.0000  minutes
